### LLM Module 1

In [1]:
from openai import OpenAI
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
client = OpenAI()

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

### Q1. Running Elastic

# Run the docker command in the terminal

docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3

In [4]:
es_client = Elasticsearch('http://localhost:9200') 

In [10]:
es_client.info()['version']['build_hash']

'42f05b9372a9a4a470db3b52817899b99a76ee73'

### Q2. Indexing the data

In [11]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [13]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

100%|██████████| 948/948 [00:20<00:00, 46.45it/s]


In [14]:
es_client.count()

ObjectApiResponse({'count': 948, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

### Q3. Searching

In [24]:
query = 'How do I execute a command in a running docker container?'
search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

response = es_client.search(index=index_name, body=search_query)
response['hits']['hits'][0]['_score']

84.050095

### Q4. Filtering

In [25]:
response['hits']['hits'][2]['_source']['question']


'How do I copy files from a different folder into docker container’s working directory?'

### Q5. Building a prompt

In [26]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                        }
                    },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                        }
                    }
                }
            }
        }

    response = es_client.search(index=index_name, body=search_query)
    
    respond_doc = []
    for hit in response['hits']['hits']:
        respond_doc.append(hit['_source'])
        
    return respond_doc

In [27]:
def build_prompt(query, context):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
    
    prompt = prompt_template.format(question=query, context=context)
    return prompt

In [28]:
query = "How do I execute a command in a running docker container?"
respond_doc = elastic_search(query)

In [30]:
context = ""
context_template = """
Q: {question}
A: {text}
""".strip()

for doc in respond_doc:
    context += (context_template.format(question=doc['question'], text=doc['text'])+"\n\n")

context = context.strip()

In [41]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [31]:
prompt = build_prompt(query, context)
len(prompt)

1462

### Q6. Tokens

In [33]:
import tiktoken

In [34]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [36]:
len(encoding.encode(text=prompt))

322